In [1]:
from collections import defaultdict
import re
import pickle
import os
import random
import numpy as np

## Shard doordash data by category

This is basically a manual map-reduce. First for each shard, we separate by the category name, then we gather entries across shards by category.

In [2]:
def get_by_category(shard):
    datafile = '/home/doordash/doordash_{}.pkl'.format(shard)
    with open(datafile, 'rb') as f:
        print("Opened {}".format(datafile))
        json = pickle.load(f)
        all_entries = []
        for entry in json:
            title = entry['title']
            section = entry['metadata']['section']
            description = entry['metadata']['description']
            category = entry['metadata'].get('restaurant_category', None)
            all_entries.append((title, description, section, category))
    print("Loaded all entries")
    category_to_entries = defaultdict(list)
    for entry in all_entries:
        og_category = entry[3]
        if og_category is None:
            og_category = 'None'
        if og_category == '':
            og_category = 'blank'
        format_category = re.sub(r"\s+", '-', og_category)
        format_category = re.sub(r"\&", "and", format_category)
        format_category = format_category.lower()
        # (title, description) in a list
        category_to_entries[format_category].append((entry[0], entry[1]))
    print("Made dictionary")
    for category, entries in category_to_entries.items():
        print(category, len(entries))
        with open(
            '/home/doordash-sharded/doordash_{}_{}'.format(category, shard), 'wb') as f:
            pickle.dump(entries, f)

In [ ]:
for i in range(0, 16):
    get_by_category(i)

In [7]:
category_to_filenames = defaultdict(list)
rootdir = '/home/doordash-sharded'
for filename in os.listdir(rootdir):
    _, category, shard = filename.split("_")
    category_to_filenames[category].append(
        os.path.join(rootdir, filename))

for category, filenames in category_to_filenames.items():
    all_items = []
    for filename in filenames:
        with open(filename, 'rb') as f:
            all_items.extend(pickle.load(f))
    with open('/home/doordash-by-category/{}'.format(category), 'wb') as f:
        pickle.dump(all_items, f)

In [9]:
from collections import Counter
category_items_counter = Counter()
for filename in os.listdir('/home/doordash-by-category'):
    with open(os.path.join('/home/doordash-by-category', filename),
              'rb') as f:
        all_items = pickle.load(f)
        category_items_counter[filename] = len(all_items)

In [10]:
print(category_items_counter.most_common())

[('asian', 1888513), ('american', 1562955), ('pizza', 961731), ('mexican', 959707), ('chinese', 642722), ('sandwiches', 642297), ('pickup', 626122), ('breakfast', 600278), ('italian', 555576), ('desserts', 555059), ('none', 506663), ('fast-food', 472846), ('sushi', 450214), ('chicken', 446070), ('burgers', 362240), ('indian', 343348), ('delis', 296270), ('coffee', 276429), ('seafood', 275364), ('japanese', 206353), ('blank', 200247), ('salad', 186696), ('thai', 185039), ('lunch', 160028), ('latin-american', 158533), ('peruvian', 155822), ('barbecue', 129460), ('greek', 128941), ('vietnamese', 119869), ('middle-eastern', 103537), ('mediterranean', 91260), ('coffee-and-tea', 87063), ('other', 75369), ('steak', 66718), ('convenience', 65039), ('vegetarian', 58239), ('grocery', 52181), ('food-and-drink', 51103), ('korean', 50348), ('caribbean', 46851), ('healthy', 46057), ('vegan', 40141), ('soup', 37458), ('halal', 35615), ('comfort-food', 34467), ('noodles', 33164), ('catering', 33024), 

# Take sharded data and clean

In [354]:
cuisine = 'indian'
with open(os.path.join('/home/doordash-by-category', cuisine),
              'rb') as f:
    all_items = pickle.load(f)
    all_titles = [e[0] for e in all_items]

In [248]:
re.sub("[\(\[].*?[\)\]]", "", 'Bacon & Sausage (2 Each)')
re.sub(r"\&", "and", "Bacon & Sausage (2 Each)")

'Bacon and Sausage (2 Each)'

In [355]:
random.choices(all_items, k=10)

[('Chaat Samosa',
  'Crispy patience stuffed with potatoes, peas mixed with special blend of spices and garbanzo beans.'),
 ('Mango Lassi', ''),
 ('Plain Steamed Rice', 'Plain Indian basmati rice.'),
 ('Cheese', ''),
 ('Lamb Keema Sizzler',
  'A spicy Pakistani minced lamb dish cooked with spices, green chili, and ginger. Served on a hot plate.'),
 ('Chicken Pudina Kabab',
  'Boneless juicy chunks of chicken marinated in mint and spices, grilled in clay oven.'),
 ('Lamb Saag', 'Lamb cooked in creamy spinach sauce.'),
 ('Garlic Naan', 'Topped with fresh garlic and cilantro.'),
 ('Ice Cream', ''),
 ('Mutter Paneer',
  'Vegetarian. Cottage cheese and green peas. Tomato based gravy, lazeez herbs.')]

In [356]:
def check_valid(title):
    check = "".join(title.split())  # remove whitespace
    check = check.replace("-", "")  # remove -, allowed
    return check.isalpha()

In [357]:
def format_valid(title):
    title = " ".join(title.split())  # whitespace to " "
    title = title.rstrip().lstrip()
    title = title.replace("-", " ")
    return title

In [358]:
letter_items = [format_valid(x[0]) for x in all_items if check_valid(x[0])]

In [361]:
len([x for x in letter_items if x==" "])

0

In [312]:
len(letter_items), len(all_items)

(1137599, 1562955)

In [ ]:
random.choices(letter_items, k=10)

## Take sharded data and tokenize

In [113]:
from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained(
    'bert-base-multilingual-cased',
    do_lower_case=True)

##### Demonstrate with particular cuisine

In [430]:
cuisine = 'american'
with open(os.path.join('/home/doordash-by-category', cuisine),
              'rb') as f:
    all_items = pickle.load(f)

The history saving thread hit an unexpected error (OperationalError('database or disk is full')).History will not be written to the database.


In [431]:
titles = [e[0] for e in all_items]
print(titles[:100])

['Extra Sauce Individual', 'Extra Onions', 'Extra Pickles', 'Extra Jalapenos', 'Extra Sauce for the Group.', '2 Pieces Catfish 4 Wings', '6 Pice whole wings with fries and bread', 'Catfish Fillets (3 Pieces)', 'Mac Cheese Bites', 'T Bone Steak w/ Potatoes, Texas Toast & Salad', 'Mushrooms', '3 Pieces Whole Wings w/ Fries', '10 Pice whole wings with fries and Bread', '20 Pieces Whole Wings w/ Fries', '30 Pieces Whole Wings w/ Fries', '40 Pieces Whole Wings w/ Fries', '50 Pieces Whole Wings w/ Fries', '100 Pieces Whole Wings w/ Fries', 'Chicken Gizzards', '10 Chicken Nuggets', '5 Pieces Chicken Tenders', '10 Pieces Drummies', '10 Pieces Chicken Boneless', 'Chicken kabab sticks with Rice', 'Gyro', 'TERIYAKI Chicken', 'Chicken', 'Steak', 'Jerk Chicken', 'Curry Chicken', 'Beef', 'Chicken', 'Gyro', 'Philly Steak', 'Italian Beef Nacho', 'Gyro Cheese Fries', 'Falafel sandwich with fries', 'Turkey burgers', 'Crispy chicken sandwich', 'Juicy Lucy Burger', 'Gyro', 'Cheese Burger', '6â\x80\x9dPhil

In [136]:
max_seq_length=64
tokenized_output = tokenizer.batch_encode_plus(
    titles[:5], max_seq_length=max_seq_length,
    pad_to_max_length=False)
print(tokenized_output)

{'input_ids': [[101, 19868, 11731, 10419, 16080, 102], [101, 19868, 30231, 13326, 102], [101, 19868, 36833, 11268, 102], [101, 19868, 10201, 47263, 58286, 102], [101, 19868, 11731, 10419, 10142, 10105, 11795, 119, 102]], 'token_type_ids': [[0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0], [0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1], [1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1]]}


###### Getting max seq length

In [34]:
cuisine_to_lens = {}
for cuisine in os.listdir('/home/doordash-by-category'):
    if os.path.isdir('/home/doordash-by-category/{}'.format(cuisine)):
        continue
    print(cuisine)
    with open(os.path.join('/home/doordash-by-category', cuisine),
                  'rb') as f:
        all_items = pickle.load(f)
    titles = [e[0] for e in all_items]
    random.shuffle(titles)
    tokenized_output = tokenizer.batch_encode_plus(
        titles[:1000], max_seq_length=128,
        pad_to_max_length=False)
    lens = [len(input_ids) for input_ids in tokenized_output['input_ids']]
    cuisine_to_lens[cuisine] = lens

sandwiches
deals
ethiopian
pickup
soup
smoothie
good-for-groups
alcohol-stores
beauty
exclusive
tapas
thai
german
cafes
pho
alcohol
belgian
indian
mediterranean
seafood
vietnamese
catering
mexican
greek
comfort-food
irish
european
ramen
heritage
bakery
newly-added
french
pakistani
takeout
desserts
halal
chicken
bubble-tea
drinks
kwb
fondue
argentine
steak
pasta
cheese
italian
late-night
japanese
south-asian
chinese
peruvian
asian
breakfast
coffee-and-tea
southern
healthy
other
cajun
brazilian
lunch
russian
none
pizza
delis
korean
barbecue
vegetarian
grocery
caribbean
noodles
filipino
british
gastropubs
burmese
sushi
middle-eastern
vegan
blank
latin-american
convenience
poke
poutineries
coffee
kosher
convenience-exclusive
gluten-free
african
burgers
salad
fast-food
food-and-drink
american
australian
spanish


In [37]:
{c: max(l) for c, l in cuisine_to_lens.items()}

{'sandwiches': 30,
 'deals': 24,
 'ethiopian': 27,
 'pickup': 35,
 'soup': 34,
 'smoothie': 39,
 'good-for-groups': 26,
 'alcohol-stores': 25,
 'beauty': 20,
 'exclusive': 10,
 'tapas': 32,
 'thai': 36,
 'german': 37,
 'cafes': 29,
 'pho': 52,
 'alcohol': 32,
 'belgian': 24,
 'indian': 24,
 'mediterranean': 23,
 'seafood': 39,
 'vietnamese': 39,
 'catering': 44,
 'mexican': 26,
 'greek': 33,
 'comfort-food': 39,
 'irish': 27,
 'european': 37,
 'ramen': 19,
 'heritage': 17,
 'bakery': 31,
 'newly-added': 19,
 'french': 32,
 'pakistani': 23,
 'takeout': 24,
 'desserts': 24,
 'halal': 29,
 'chicken': 27,
 'bubble-tea': 33,
 'drinks': 33,
 'kwb': 27,
 'fondue': 24,
 'argentine': 39,
 'steak': 32,
 'pasta': 30,
 'cheese': 29,
 'italian': 28,
 'late-night': 22,
 'japanese': 35,
 'south-asian': 28,
 'chinese': 34,
 'peruvian': 27,
 'asian': 24,
 'breakfast': 22,
 'coffee-and-tea': 27,
 'southern': 25,
 'healthy': 25,
 'other': 20,
 'cajun': 29,
 'brazilian': 28,
 'lunch': 22,
 'russian': 24,


### Saving tokenization

In [387]:
rootdir = '/home/doordash-by-category/cleaned-tokenized/bert-base-multilingual-cased'
max_seq_length = 64
assert os.path.exists(rootdir)

In [388]:
file_pattern = "titles_{cuisine}_{max_seq_len}_{input_key}"

In [389]:
do_clean = True

def format_valid(title):
    title = " ".join(title.split())  # whitespace to " "
    title = title.rstrip().lstrip()
    title = title.replace("-", " ")
    return title

def check_valid(title):
    check = "".join(title.split())  # remove whitespace
    check = check.replace("-", "")  # remove -, allowed
    return check.isalpha()

In [393]:
targets_to_categories = {
    # Label to list of filenames corresponding to that label
    'mexican': ['mexican'],
    'chinese': ['chinese'],
    'american': ['american', 'burgers'],
    'italian': ['italian', 'pasta'],
    'thai': ['thai'],
    'indian': ['indian'],
    'japanese': ['japanese', 'ramen', 'sushi'],
    'other': ['african', 'alcohol', 'argentine', 'australian',
             'bakery', 'belgian', 'brazilian', 'burmese', 'desserts',
             'drinks', 'ethiopian', 'filipino', 'french',
             'german', 'greek', 'korean', 'vietnamese', 'poke']
}
all_categories = [item for sublist in targets_to_categories.values() for item in sublist]

In [394]:
print(max_seq_length)

64


In [397]:
file_pattern = "titles_{cuisine}_{max_seq_len}_{input_key}"
lens = {}
for cuisine in all_categories:# os.listdir('/home/doordash-by-category'):
    if os.path.isdir('/home/doordash-by-category/{}'.format(cuisine)):
        # Not a real cuisine, for example bert-base-multilingual-cased is a folder containing tokenizations
        continue
    with open(os.path.join('/home/doordash-by-category', cuisine), 'rb') as f:
        all_items = pickle.load(f)
    titles = [e[0] for e in all_items]
    if do_clean:
        titles = [format_valid(t) for t in titles if check_valid(t)]
    
    lens[cuisine] = len(titles)
    if os.path.exists(os.path.join(
        rootdir, file_pattern.format(
            cuisine=cuisine, max_seq_len=max_seq_length, 
            input_key="input_ids"))):
        # Tokenziation already exists in the given folder
        continue

    print(cuisine, len(titles))
    tokenized_output = tokenizer.batch_encode_plus(
        titles,
        max_length=max_seq_length,
        pad_to_max_length=True)
    
    print("Done tokenizing")
    for input_key, arr in tokenized_output.items():
        filename = os.path.join(rootdir,
                                file_pattern.format(cuisine=cuisine,
                                      max_seq_len=max_seq_length,
                                      input_key=input_key))
        arr = np.array(arr)
        fp = np.memmap(filename, dtype='int64', mode='w+',
                      shape=arr.shape)
        fp[:] = arr[:]
        del fp

In [398]:
with open(os.path.join(rootdir, 'lens.pkl'), 'wb') as f:
    # Keep lens which is needed for loading a memmap
    pickle.dump(lens, f)

# Dataloader

In [399]:
import torch
import itertools
from collections import Counter, OrderedDict, defaultdict
from torch.utils.data import DataLoader

In [400]:
class IterableTitles(torch.utils.data.IterableDataset):
    def __init__(self, cuisine, root_dir, max_seq_length, len_dict):
        super(IterableTitles).__init__()
        self.cuisine = cuisine
        self.max_seq_length = max_seq_length
        self.datafile_pattern = os.path.join(
            root_dir, 
            'titles_{}_{}_{{}}'.format(cuisine, max_seq_length))
        self.shape = (len_dict[cuisine], max_seq_length)
        self.length = len_dict[cuisine]

    def __len__(self):
        return self.length

    def __iter__(self):
        input_key_to_memmap = {}
        for input_key in ['input_ids',
                          'attention_mask',
                         'token_type_ids']:
            datafile = self.datafile_pattern.format(input_key)
            fp = np.memmap(
                datafile, dtype='int64', mode='r+',
                shape=self.shape)
            input_key_to_memmap[input_key] = fp
        while True:
            i = random.choice(range(self.shape[0]))
        #for i in itertools.cycle(range(self.shape[0])):
            yield (
                input_key_to_memmap['input_ids'][i],
                input_key_to_memmap['token_type_ids'][i],
                input_key_to_memmap['attention_mask'][i])


class MultiStreamDataLoader:
    def __init__(self, targets_to_categories, batch_size, rootdir):
        self.targets = list(sorted(targets_to_categories.keys()))
        self.targets_to_categories = targets_to_categories
        
        self.categories_to_target = OrderedDict()
        self.targets_to_categories_weights= {}
        self.categories_to_dataset = {}
        self.categories_to_dataset_iter = {}
        with open(os.path.join(rootdir, 'lens.pkl'), 'rb') as f:
            self.categories_to_len = pickle.load(f)
        
        for t, list_of_c in targets_to_categories.items():
            for c in list_of_c:
                self.categories_to_target[c] = t
                dataset = IterableTitles(c, rootdir, 64, self.categories_to_len)
                self.categories_to_dataset[c] = dataset
                self.categories_to_dataset_iter[c] = iter(DataLoader(dataset, batch_size=None))

        for t, c_list in self.targets_to_categories.items():
            self.targets_to_categories_weights[t] = [self.categories_to_len[c] for c in c_list]

        self.batch_size = batch_size
        self.total_samples = sum(self.categories_to_len.values())


    def __len__(self):
        return self.total_samples//self.batch_size
    

    def __iter__(self):
        while True:
            buffer = []
            labels = []
            target_choices = random.choices(self.targets, k=self.batch_size)
            category_choices = []
            for t in target_choices:
                category_choices.append(random.choices(
                    self.targets_to_categories[t],
                    weights=self.targets_to_categories_weights[t],
                    k=1)[0])

            category_counter = Counter(category_choices)
            category_labels = []
            for category, num_sample in category_counter.items():
                category_labels.extend([category for _ in range(num_sample)])
                l_num = self.targets.index(self.categories_to_target[category])
                labels.extend([l_num for _ in range(num_sample)])
                buffer.extend(
                    [next(self.categories_to_dataset_iter[category]) for _ in range(num_sample)]
                )
            yield (torch.stack([b[0] for b in buffer]),
                   torch.stack([b[1] for b in buffer]),
                   torch.stack([b[2] for b in buffer]),
                   torch.tensor(labels),
                   category_labels
                  )

In [401]:
rootdir = '/home/doordash-by-category/cleaned-tokenized/bert-base-multilingual-cased/'

In [425]:
targets_to_categories = {
    # Label to list of filenames corresponding to that label
    'mexican': ['mexican'],
    'chinese': ['chinese'],
    'american': ['american', 'burgers'],
    'italian': ['italian', 'pasta'],
    'thai': ['thai'],
    'indian': ['indian'],
    'japanese': ['japanese', 'ramen', 'sushi'],
    'other': ['african', 'argentine', 'australian',
             'bakery', 'belgian', 'brazilian', 'burmese', # 'desserts',
             'drinks', 'ethiopian', 'filipino', 'french', # 'alcohol'
             'german', 'greek', 'korean', 'vietnamese', 'poke']
}

ds = MultiStreamDataLoader(targets_to_categories,
                           batch_size=800,
                           rootdir=rootdir)

In [426]:
# ds.targets

In [427]:
# ds.targets_to_categories_weights

In [428]:
[(t, sum(w)) for t, w in ds.targets_to_categories_weights.items()]

[('mexican', 703520),
 ('chinese', 246222),
 ('american', 1374937),
 ('italian', 420704),
 ('thai', 111192),
 ('indian', 272184),
 ('japanese', 440474),
 ('other', 253855)]

In [429]:
for batch_num, batch in enumerate(ds):
    if batch_num == 5:
        break
    print(Counter(batch[3].numpy()))
    input_ids = batch[0]
    labels = batch[3]
    categories = batch[4]
    for i, ids in enumerate(input_ids):
        print(
            ds.targets[labels[i]], "({})".format(categories[i]),
            ":",
            tokenizer.convert_tokens_to_string(
                tokenizer.convert_ids_to_tokens(ids, skip_special_tokens=True)),
            # "\n",
            # ids
        )
    print("=====")

Counter({0: 114, 2: 108, 5: 107, 1: 103, 3: 99, 4: 93, 6: 91, 7: 85})
mexican (mexican) : steak cemita with onions
mexican (mexican) : quesadilla
mexican (mexican) : mini burrito
mexican (mexican) : grilled cazuelada
mexican (mexican) : oreo cake
mexican (mexican) : pechuga a la plancha combo
mexican (mexican) : tacos de nopales
mexican (mexican) : enchiladas suizas
mexican (mexican) : bistec a la mexicana
mexican (mexican) : grilled cactus nopal
mexican (mexican) : came asada
mexican (mexican) : stuffed steak taco dinner
mexican (mexican) : arroz con pollo
mexican (mexican) : blackberry izze
mexican (mexican) : bottled water
mexican (mexican) : quesadilla bbq
mexican (mexican) : chorizo burrito
mexican (mexican) : brisket
mexican (mexican) : especial united
mexican (mexican) : media parrillada de mariscos
mexican (mexican) : bean taco
mexican (mexican) : picadillo
mexican (mexican) : milanesa
mexican (mexican) : two tostadas
mexican (mexican) : taco ensalada
mexican (mexican) : toro y

italian (italian) : fettuccine con gamberi
italian (italian) : pinwheel antipasto
italian (italian) : pecan crusted chicken with grilled zucchini
italian (italian) : calamari fritti
italian (italian) : chicken caesar salad
italian (italian) : baked ravioli with meatballs
italian (italian) : juices
italian (italian) : linguini with scallops
italian (italian) : baccala alla livornese
italian (italian) : baby spinach
italian (italian) : fettuccine
italian (italian) : shrimp cocktail
italian (italian) : the office jumbo wings
italian (italian) : staciatela alaromana
italian (italian) : pizza burger
italian (italian) : cheeseburger
italian (italian) : insalata di cesare
italian (italian) : grilled chicken sandwich
italian (italian) : gluten free top it off pizza
italian (italian) : breaded zucchini
italian (italian) : manicotti with sausage
italian (italian) : pint of meat sauce
italian (italian) : shrimp fra diavolo
italian (italian) : veal parmesan
italian (italian) : milk
italian (italia

indian (indian) : lamb dansak
indian (indian) : papad
indian (indian) : mint kulcha
indian (indian) : paneer pakora
indian (indian) : special naan kadhmiri with rosemary
indian (indian) : beef curry
indian (indian) : payasum
indian (indian) : gulab jamun
indian (indian) : chana bhuna
indian (indian) : aloo mutter
indian (indian) : baigan bhartha
indian (indian) : malai kofta
indian (indian) : soda
indian (indian) : biryani
indian (indian) : veg dum biryani
indian (indian) : vanjaram fish fry
indian (indian) : chili chicken parotta
indian (indian) : cheese uthappam
indian (indian) : veg kothu
indian (indian) : french fries
indian (indian) : tawa roti
indian (indian) : vegetable biriyani
indian (indian) : mulligatawny soup
indian (indian) : mughlai mutton roast
indian (indian) : the bianca
indian (indian) : kadhai chicken
indian (indian) : salmon tikka
indian (indian) : vegetarian lunch box
indian (indian) : samosa chaat
indian (indian) : shrimp palak
indian (indian) : veggie biryani
ind

Counter({1: 110, 5: 104, 6: 103, 2: 102, 0: 100, 4: 97, 7: 96, 3: 88})
indian (indian) : fish curry
indian (indian) : lobster vindaloo
indian (indian) : nilgiri paneer tikka shashlik
indian (indian) : bullet naan
indian (indian) : paneer korma
indian (indian) : vegetable noodles
indian (indian) : veg fried rice
indian (indian) : chicken tikka masala
indian (indian) : kheer
indian (indian) : egg fried rice
indian (indian) : goat achari
indian (indian) : special onion rava masala with cheese
indian (indian) : chicken saag
indian (indian) : beef achaar
indian (indian) : lamb saag
indian (indian) : murgh saagwala
indian (indian) : lemon pickle
indian (indian) : lamb vindaloo
indian (indian) : curd rice
indian (indian) : pappadam
indian (indian) : fish coconut curry
indian (indian) : saag paneer
indian (indian) : chicken tikka
indian (indian) : tadka daal
indian (indian) : andhra ghee roast
indian (indian) : chicken tikka salad
indian (indian) : chicken korma
indian (indian) : cheese dosa
i

thai (thai) : yum tao hu salad
thai (thai) : pineapple fried rice
thai (thai) : ruby black tea
thai (thai) : genmai green tea
thai (thai) : moroccan breakfast
other (vietnamese) : banana
other (vietnamese) : coke
other (vietnamese) : yellow curry
other (vietnamese) : pad garlic
other (vietnamese) : pad thai
other (vietnamese) : sardine sandwich
other (vietnamese) : san pellegrino orange
other (vietnamese) : drunken noodle
other (vietnamese) : choice of shrimp or beef or combination
other (vietnamese) : side of sticky rice
other (vietnamese) : okdol bibimbob
other (vietnamese) : chips
other (vietnamese) : com suon bi dau hu ky
other (vietnamese) : grilled chicken sandwich
other (vietnamese) : red trolley
other (vietnamese) : almond shrimp
other (vietnamese) : pad thai
other (vietnamese) : jumbo combo rice dish
american (american) : fritos pie stack
american (american) : gluten free pancakes
american (american) : spinach
american (american) : tampiquena
american (american) : mashed potat

...

american (american) : fridays signature whiskey glazed burger
american (american) : fried calamari
american (american) : lite side
american (american) : spartan salad
chinese (chinese) : cashew nut chicken
chinese (chinese) : green pepper steak
chinese (chinese) : deep fried fillet rock cod with sweet corn sauce
chinese (chinese) : satay beef chow fun
chinese (chinese) : roast pork chop suey
chinese (chinese) : bottled drinks
chinese (chinese) : house fried rice
chinese (chinese) : curry beef
chinese (chinese) : beef with mixed vegetables
chinese (chinese) : vegetable
chinese (chinese) : sizzling spicy chicken
chinese (chinese) : tilapia fillet
chinese (chinese) : twice cooked pork
chinese (chinese) : mapo tofu
chinese (chinese) : sprite
chinese (chinese) : dry garlic pork
chinese (chinese) : kung pao tofu
chinese (chinese) : kung pao prawns
chinese (chinese) : veggie burger
chinese (chinese) : canned soda
chinese (chinese) : deep fried tofu szechuan style
chinese (chinese) : pepper be

other (korean) : korean soft drinks
japanese (japanese) : coke in can
japanese (japanese) : vegetable soup
japanese (japanese) : squid
japanese (japanese) : fried calamari
japanese (japanese) : toro sushi
japanese (japanese) : fried calamari
japanese (japanese) : gnart
japanese (japanese) : chicken katsu ramen
japanese (japanese) : orange juice
japanese (japanese) : organic brown rice
japanese (japanese) : karage curry
japanese (japanese) : edamame
japanese (japanese) : et bowl
japanese (japanese) : sweet potato tempura roll
japanese (japanese) : peanut avocado roll
japanese (japanese) : perrier
japanese (japanese) : large chicken fried rice
japanese (japanese) : honeydew
japanese (japanese) : vegetable with noodle
japanese (japanese) : edamame
japanese (japanese) : crispy red snapper
japanese (japanese) : viking roll
japanese (japanese) : seafood dynamite
japanese (japanese) : edamame
japanese (japanese) : yakisoba
japanese (japanese) : pork fried rice
japanese (japanese) : vegetable


indian (indian) : paneer butter masala
indian (indian) : butter naan
indian (indian) : gulab jamun
indian (indian) : saffron lamb medallions
indian (indian) : sparkling water
indian (indian) : lentil soup
indian (indian) : papadum
indian (indian) : fish saagwala
indian (indian) : shrimp
indian (indian) : onion chilly cheese uttapam
indian (indian) : dal tarka
indian (indian) : kesar lassi
indian (indian) : malai chicken tikka
indian (indian) : garlic naan
indian (indian) : mapo tofu
indian (indian) : shrimp jalefrazie
indian (indian) : mixed dal
indian (indian) : pickle
indian (indian) : chicken manchurian
indian (indian) : chicken pepper fry
indian (indian) : lamb akbari
indian (indian) : baigan bharta
indian (indian) : tandoori roti
indian (indian) : shrimp biryani
indian (indian) : vegetable manchurian
indian (indian) : chilli chicken
indian (indian) : house
indian (indian) : bhindi masala
indian (indian) : chicken samosa
indian (indian) : vegetable biryani
indian (indian) : chicken

chinese (chinese) : roast duck rice noodle
chinese (chinese) : beef with mixed vegetables
chinese (chinese) : beef with mixed vegetable
chinese (chinese) : dragon and phoenix
chinese (chinese) : breaded chicken burger
chinese (chinese) : mango pudding
chinese (chinese) : chicken in spicy black bean sauce
chinese (chinese) : string beans
chinese (chinese) : royal signature eggplant salad
chinese (chinese) : crispy shrimp
chinese (chinese) : tofu with garlic sauce
chinese (chinese) : chicken garlic sauce
chinese (chinese) : chow mein with bean sprouts and dark soy sauce
chinese (chinese) : curry shrimp
chinese (chinese) : buffalo chicken wrap
chinese (chinese) : szechuan chicken
chinese (chinese) : can soda
thai (thai) : watermelon
thai (thai) : oriental beauty oolong tea
thai (thai) : winermelon tea with grass jelly
thai (thai) : garlic and pepper
thai (thai) : stir fried mixed vegetable
thai (thai) : chicken wings
thai (thai) : franch fries with garlic pepper
thai (thai) : iced strawbe

other (poke) : cali ahi poke
other (poke) : roasted oolong tea
other (poke) : hawaii
other (german) : veal jager schnitzel
other (african) : jollof rice
other (african) : frozen hot chocolate
other (ethiopian) : cappuccino
other (ethiopian) : mixed vegetables
other (ethiopian) : yasa awaze tibs
other (brazilian) : flan
other (brazilian) : bread on the plate
other (australian) : pasta with mixed vegetables
japanese (ramen) : rib eye beef with teriyaki sauce bento box
other (argentine) : ensalata cesar
=====
